# File Name: advanced_image_patterns_part2.ipynb
### Location: Chapter 18
### Purpose: 
#####       1. Image Inpainting
#####       2. Image Conditioning
#####       3. Colour Conditioning
#####       4. Image Outpainting
#####       5. Background Removal 
#####       6. Combination of Text and Image

##### Dependency: simple-sageMaker-bedrock.ipynb at Chapter 3 should work properly.
# <ins>-----------------------------------------------------------------------------------</ins>

# <ins>Amazon SageMaker Classic</ins>
#### Those who are new to Amazon SageMaker Classic. Follow the link for the details. https://docs.aws.amazon.com/sagemaker/latest/dg/studio.html

# <ins>Environment setup of Kernel</ins>
##### Fill "Image" as "Data Science"
##### Fill "Kernel" as "Python 3"
##### Fill "Instance type" as "ml-t3-medium"
##### Fill "Start-up script" as "No Scripts"
##### Click "Select"

###### Refer https://docs.aws.amazon.com/sagemaker/latest/dg/notebooks-create-open.html for details.

# <ins>Mandatory installation on the kernel through pip</ins>

##### This lab will work with below software version. But, if you are trying with latest version of boto3, awscli, and botocore. This code may fail. You might need to change the corresponding api. 

##### You will see pip dependency errors. you can safely ignore these errors and continue executing rest of the cell. 

In [ ]:
%%time 

%pip install --no-build-isolation --force-reinstall -q \
    "boto3" \
    "awscli" \
    "botocore" \
    "utils" \
    "matplotlib" \
    "sagemaker" \
    "numpy<2"

# <ins>Disclaimer</ins>

##### You will see pip dependency errors. you can safely ignore these errors and continue executing rest of the cell.

# <ins>Restart the kernel</ins>

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

# <ins>Python package import</ins>

##### boto3 offers various clients for Amazon Bedrock to execute various actions.
##### botocore is a low-level interface to AWS tools, while boto3 is built on top of botocore and provides additional features

In [ ]:
import os
import sys
import json
import io
import base64
import random
import warnings
import boto3
import botocore
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg 
from PIL import Image, ImageOps
import sagemaker

### Ignore warning 

In [ ]:
warnings.filterwarnings('ignore')

## Define important environment variable

In [ ]:
# Try-except block to handle potential errors during execution
try:
    # Create a new Boto3 session to interact with AWS services
    # This session manages credentials and region configuration for AWS interactions
    boto3_session = boto3.session.Session()

    # Retrieve the current AWS region from the session (e.g., 'us-east-1', 'us-west-2')
    aws_region_name = boto3_session.region_name

    # Initialize Bedrock and Bedrock Runtime clients using Boto3
    # These clients enable interactions with AWS Bedrock-related services
    boto3_bedrock_client = boto3.client('bedrock', region_name=aws_region_name)
    boto3_bedrock_runtime_client = boto3.client('bedrock-runtime', region_name=aws_region_name)

    # Create a SageMaker session and retrieve the execution role ARN
    # The role ARN authorizes SageMaker to perform tasks on behalf of the user
    sagemaker_role_arn = sagemaker.get_execution_role()

    # Specify the Amazon Titan image generator model ID for multimodal processing
    amazon_titan_image_model_id = "amazon.titan-image-generator-v2:0"

    # Specify the Amazon Titan embedding model ID for multimodal indexing
    multimodal_embed_model_id = "amazon.titan-embed-image-v1"

    # Store all relevant variables in a dictionary for easier access and management
    variables_store = {
        "aws_region_name": aws_region_name,                           # AWS region name
        "boto3_bedrock_client": boto3_bedrock_client,                 # Bedrock client instance
        "boto3_bedrock_runtime_client": boto3_bedrock_runtime_client, # Bedrock Runtime client instance
        "boto3_session": boto3_session,                               # Current Boto3 session object
        "sagemaker_role_arn": sagemaker_role_arn,                     # SageMaker execution role ARN
        "multimodal_embed_model_id": multimodal_embed_model_id,       # Titan embedding model ID
        "amazon_titan_image_model_id": amazon_titan_image_model_id    # Titan image generator model ID
    }

    # Print all stored variables for debugging and verification
    for var_name, value in variables_store.items():
        print(f"{var_name}: {value}")

# Handle any exceptions that occur during the execution
except Exception as e:
    # Print an error message if an unexpected error occurs
    print(f"An unexpected error occurred: {e}")


# Common code 

### The provided Python code allows you to plot multiple images side by side with corresponding headings.

In [ ]:
%%time
def plot_multiple_images(images, headings, cols=3):
    """
    Plots multiple images side by side with headings.

    Parameters:
    images (list): A list of image paths.
    headings (list): A list of headings for each image.
    cols (int): The number of columns to display the images in (default is 3).
    """
    # Calculate number of rows needed
    rows = (len(images) + cols - 1) // cols

    # Create a figure and axes
    fig, axes = plt.subplots(rows, cols, figsize=(cols * 5, rows * 5))

    # Flatten axes array if there are multiple rows
    axes = axes.flatten()

    # Loop through images and plot them
    for i in range(len(images)):
        # Read the image from file path
        img = mpimg.imread(images[i])
        
        # Plot the image
        axes[i].imshow(img)
        axes[i].axis('off')  # Hide axis
        
        # Add the heading
        axes[i].set_title(headings[i], fontsize=14, weight='bold')

    # Hide unused axes if the number of images is less than the number of axes
    for i in range(len(images), len(axes)):
        axes[i].axis('off')

    # Adjust layout
    plt.tight_layout()
    plt.show()

# Example usage
# plot_multiple_images(images, headings, cols=2)

### base64 encoded string of the image

In [ ]:
%%time
def encode_image(image_path):
    # Open the image file in binary mode
    with open(image_path, "rb") as image_file:
        # Encode the image to base64
        encoded_image = base64.b64encode(image_file.read()).decode("utf-8")
    return encoded_image

# Example usage
## encoded_image = encode_image(image_path)

### Decode base64 to image and save

In [ ]:
%%time

def decode_image(encoded_image_b64, save_path):
    """
    Decodes a base64-encoded image and saves it to a specified file location.

    Args:
        encoded_image_b64 (str): The base64-encoded string of the image.
        save_path (str): The path where the decoded image will be saved.

    Returns:
        None
    """
    try:
        # Decode the base64 image and open it as a PIL Image
        decoded_image = Image.open(
            io.BytesIO(
                base64.decodebytes(bytes(encoded_image_b64, "utf-8"))
            )
        )
        
        # Save the decoded image to the specified path
        decoded_image.save(save_path)
        print(f"Image successfully saved to {save_path}")
    except (base64.binascii.Error, IOError) as e:
        print(f"Error decoding or saving the image: {e}")
        raise  # Optionally re-raise the exception for external handling

# Example usage
# decode_image(generated_image_b64, "output_image.png")


### The function invoke_bedrock_model is designed to call an Amazon Bedrock model for image generation, with robust error handling and clear feedback mechanisms. It accepts a Boto3 runtime client, a JSON payload (body), and an optional model ID

In [ ]:
%%time
def invoke_bedrock_model(boto3_bedrock_runtime_client, body, model_id):
    """
    Invokes the Amazon Bedrock model for image generation with error handling.
    
    Args:
        boto3_bedrock_runtime_client: The Boto3 Bedrock runtime client.
        body (str): The request payload as a JSON string.
        model_id (str): The model identifier to invoke (default: "amazon.titan-image-generator-v1").
    
    Returns:
        str: The base64-encoded image data if successful.
    """
    try:
        # Model invocation
        response = boto3_bedrock_runtime_client.invoke_model(
            body=body,
            modelId=model_id,
            accept="application/json", 
            contentType="application/json"
        )
        
        # Output processing
        response_body = json.loads(response.get("body").read())  # Decode response body
        img_b64 = response_body["images"][0]  # Extract the first image (base64-encoded)
        print(f"Output (truncated): {img_b64[:80]}...")  # Print truncated base64 string for debugging
        
        return img_b64  # Return the full base64-encoded image
    
    except KeyError as e:
        print(f"KeyError: Missing expected key in response - {e}")
        return None
    
    except json.JSONDecodeError as e:
        print(f"JSONDecodeError: Failed to parse response body - {e}")
        return None
    
    except boto3.exceptions.Boto3Error as e:
        print(f"Boto3Error: AWS SDK error - {e}")
        return None
    
    except Exception as e:
        print(f"Unexpected Error: {e}")
        return None

# Section 4: Image Inpainting

### You previously generated an image of a river at dusk using a prompt. Now, you want to enhance the image by adding a boat in the middle of the scene.
### The original image is located at data/generated_image/generated_image_15.png.
### To achieve this, you will use the image inpainting technique.

In [ ]:
%%time

image_inpainting_source_image = "data/generated_image/generated_image_15.png"
image_inpainting_mask_image = "data/generated_image/generated_image_mask.png"
image_inpainting_target_image = "data/generated_image/generated_image_inpainting.png"

# Read the image file and encode it in base64
inpainting_source_encoded_image = encode_image(image_inpainting_source_image)

### This script defines a function, create_inpainting_mask, to generate a segmentation mask for inpainting by filling a specified rectangular area with black and the rest with white. The function includes input validation for box coordinates to ensure proper execution. In the main execution, the script loads a source image, calculates the coordinates for centering a mask of specified dimensions, and creates the mask using the function. 

In [ ]:
%%time

def create_inpainting_mask(img, box):
    """
    Generates a segmentation mask for inpainting.

    Args:
        img (PIL.Image.Image): The input image.
        box (tuple): Coordinates for the box (left, top, right, bottom).

    Returns:
        PIL.Image.Image: A mask image with the specified box filled with black,
                         and the rest filled with white.
    """
    try:
        # Validate the input box coordinates
        assert len(box) == 4, "Box must have exactly 4 coordinates (left, top, right, bottom)."
        assert box[0] < box[2], "Left coordinate must be less than the right coordinate."
        assert box[1] < box[3], "Top coordinate must be less than the bottom coordinate."
        
        # Generate the mask
        img_size = img.size
        mask = ImageOps.expand(
            Image.new(
                mode="RGB", 
                size=(box[2] - box[0], box[3] - box[1]),  # Mask dimensions (width, height)
                color='black'  # Black region for inpainting
            ),
            border=(
                box[0],            # Left padding
                box[1],            # Top padding
                img_size[0] - box[2],  # Right padding
                img_size[1] - box[3]   # Bottom padding
            ),
            fill='white'  # White for the rest of the image
        )
        return mask
    except AssertionError as e:
        print(f"Error in box coordinates: {e}")
        raise
    except Exception as e:
        print(f"Unexpected error: {e}")
        raise

# Main execution
try:
    # Load the source image
    image = Image.open(image_inpainting_source_image)
    image_size = image.size

    # Define the size of the mask (width, height)
    mask_width = 200
    mask_height = 200

    # Calculate the coordinates for the box to center the mask
    box = (
        (image_size[0] - mask_width) // 2,  # Left
        (image_size[1] - mask_height) // 2,  # Top
        (image_size[0] + mask_width) // 2,  # Right
        (image_size[1] + mask_height) // 2   # Bottom
    )

    # Create the mask
    mask = create_inpainting_mask(image, box)

    # Display the mask (for debugging purposes)
    mask.save(image_inpainting_mask_image)

except FileNotFoundError as e:
    print(f"Image file not found: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

### Image Inpainting prompt 

In [ ]:
inpaint_prompt = "Add a boat "
negative_prompts = "Poor quality, low resolution."

In [ ]:
%%time

inpainting_source_encoded_mask_image = encode_image(image_inpainting_mask_image)

# Payload creation
body = json.dumps({
    "taskType": "INPAINTING",
    "inPaintingParams": {
        "text": inpaint_prompt,              # Optional
        "negativeText": negative_prompts,    # Optional
        "image": inpainting_source_encoded_image,      # Required
        "maskImage": inpainting_source_encoded_mask_image,  # Input maskImage based on the values 0 (black) or 255 (white) only
    },                                                 
    "imageGenerationConfig": {
        "numberOfImages": 1,
        "quality": "premium",
        "height": 1024,
        "width": 1024,
        "cfgScale": 7.5,
        "seed": 42
    }
})

In [ ]:
%%time
generated_image_b64 = invoke_bedrock_model(boto3_bedrock_runtime_client, body, amazon_titan_image_model_id )
decode_image(generated_image_b64, image_inpainting_target_image)

print("\n\n")
images = [ image_inpainting_source_image , image_inpainting_mask_image , image_inpainting_target_image ]
headings =[ "Source Image" , "Mask Image" , "Generated Image" ]
plot_multiple_images(images, headings, cols=3)
print("\n\n")

# Section 5: Image Conditioning

### You previously generated an image of an elephant in a beautifull landscape using a prompt. Now, you want to enhance the image within the photoframe.
### The original image is located at data/generated_image/generated_image_5.png.
### To achieve this, you will use the image conditioning technique.

### Image Conditioning prompt for canny edge

In [ ]:
prompt = "A photo frame with a big elephant image."

In [ ]:
image_conditioning_source_image = "data/generated_image/generated_image_5.png"
image_conditioning_target_image = "data/generated_image/generated_image_conditioning.png"

In [ ]:
%%time

conditioning_source_encoded_image = encode_image(image_conditioning_source_image)

# Generate image conditioned on reference image
body = json.dumps({
    "taskType": "TEXT_IMAGE",
    "textToImageParams": {
        "text": prompt,
        "conditionImage": conditioning_source_encoded_image,
        "controlMode": "CANNY_EDGE",
        "controlStrength": 0.7,
    },
    "imageGenerationConfig": {
        "numberOfImages": 1,
        "seed": 42,
    }
})

In [ ]:
%%time

generated_image_b64 = invoke_bedrock_model(boto3_bedrock_runtime_client, body, amazon_titan_image_model_id )
decode_image(generated_image_b64, image_conditioning_target_image)

print("\n\n")
images = [ image_conditioning_source_image , image_conditioning_target_image ]
headings =[ "Source Image" ,  "Generated Image" ]
plot_multiple_images(images, headings, cols=2)
print("\n\n")

### You previously generated an image of a photoframe with an elephant in a beautifull landscape using imae conditioning canny edge. Now, you want to enhance the image of the photoframe 
### within the photoframe placed on a wooden table. Next to the frame is a beautiful flower vase, holding a vibrant bouquet of fresh flowers.
### The original image is located at data/generated_image/generated_image_conditioning.png.
### To achieve this, you will use the image conditioning segmentation technique.

### Image Conditioning prompt for segmentation

In [ ]:
prompt = "A photo frame featuring an elegant image of an elephant, placed on a wooden table. Next to the frame is a beautiful flower vase, holding a vibrant bouquet of fresh flowers. The setting is cozy, with soft natural lighting that enhances the intricate details of the elephant artwork and the delicate flowers in the vase. The overall ambiance is serene and artistic, creating a perfect balance between nature and artistry."

In [ ]:
image_conditioning_source_image = "data/generated_image/generated_image_conditioning.png"
image_conditioning_target_image = "data/generated_image/generated_image_conditioning_refined.png"

In [ ]:
%%time

conditioning_source_encoded_image = encode_image(image_conditioning_source_image)

# Generate image condition on reference image
body = json.dumps(
    {
        "taskType": "TEXT_IMAGE",
        "textToImageParams": {
            "text": prompt,  # Required
            "conditionImage": conditioning_source_encoded_image, # Optional
            "controlMode": "SEGMENTATION", # Optional: CANNY_EDGE | SEGMENTATION
            "controlStrength": 0.6,  # Range: 0.2 to 1.0,
        },
        "imageGenerationConfig": {
                "numberOfImages": 1,
                "seed": 42,
            }
        
    }
)

In [ ]:
%%time

generated_image_b64 = invoke_bedrock_model(boto3_bedrock_runtime_client, body, amazon_titan_image_model_id )
decode_image(generated_image_b64, image_conditioning_target_image)

print("\n\n")
images = [ image_conditioning_source_image , image_conditioning_target_image ]
headings =[ "Source Image" ,  "Generated Image" ]
plot_multiple_images(images, headings, cols=2)
print("\n\n")

# Section 6: Color Conditioning

### You previously generated an image of a photoframe with an elephant in a beautifull landscape using imae conditioning segmentation. Now, you want to enhance the image of the photoframe 
### within the photoframe placed on a wooden table. Next to the frame is a beautiful flower vase, holding a vibrant bouquet of fresh flowers with a specific color coding like hex_color_code = ['#FF9900', '#232F3E', '#F2F2F2', '#000000', '#146EB4']
### The original image is located at data/generated_image/generated_image_conditioning.png.
### To achieve this, you will use the colour conditioning technique.

### Colour Conditioning prompt

In [ ]:
prompt = "A photo frame featuring of a original elephant, placed on a wooden table. Next to the frame is a beautiful flower vase. elephant is side wise."

In [ ]:
color_conditioning_source_image = "data/generated_image/generated_image_conditioning.png"
color_conditioning_target_image = "data/generated_image/generated_color_conditioning.png"

hex_color_code = ['#FF9900', '#232F3E', '#F2F2F2', '#000000', '#146EB4']

In [ ]:
%%time 

conditioning_source_encoded_image = encode_image(color_conditioning_source_image)
    
    
# Generate image condition on color palette
body = json.dumps({
    "taskType": "COLOR_GUIDED_GENERATION",
    "colorGuidedGenerationParams": {
        "text": prompt,
        "colors": hex_color_code,
        "referenceImage": conditioning_source_encoded_image,
    },
    "imageGenerationConfig": {
        "numberOfImages": 1,
        "seed": 42,
    }
})

In [ ]:
import matplotlib.pyplot as plt; plt.figure(figsize=(8, 2)); [plt.gca().add_patch(plt.Rectangle((i, 0), 1, 1, color=c)) or plt.text(i + 0.5, -0.5, c, ha='center', va='center', fontsize=10) for i, c in enumerate(['#FF9900', '#232F3E', '#F2F2F2', '#000000', '#146EB4'])]; plt.xlim(0, 5); plt.ylim(-1, 1); plt.axis('off'); plt.show()

In [ ]:
%%time

generated_image_b64 = invoke_bedrock_model(boto3_bedrock_runtime_client, body, amazon_titan_image_model_id )
decode_image(generated_image_b64, color_conditioning_target_image)

print("\n\n")
images = [ color_conditioning_source_image , color_conditioning_target_image ]
headings =[ "Source Image" ,  "Generated Image" ]
plot_multiple_images(images, headings, cols=2)
print("\n\n")

# Section 7: Image Outpainting

### You previously generated an image of a photoframe with an elephant in a beautifull landscape using color conditioning. Now, you want to enhance the image of the photoframe 
### hanging on the wall.
### The original image is located at data/generated_image/generated_color_conditioning.png.
### To achieve this, you will use the cimge outpainting technique.

### Image Outpainting prompt

In [ ]:
# Define the prompt and reference image
prompt = "A big white wall in the background, the elephant photo frame is on the wall."
mask_prompt = "elephant photo frame"

In [ ]:
image_outpainting_source_image = "data/generated_image/generated_color_conditioning.png"
image_outpainting_target_image = "data/generated_image/generated_image_outpainting.png"

# Expansion setting
target_width = 1024
target_height = 1024
horizontal_position_percent=0.3
vertical_position_percent=0.5

In [ ]:
%%time

# Load reference image
original_image = Image.open(image_outpainting_source_image)
original_width, original_height = original_image.size

# Calculate the position of the original image on the expanded canvas.
position = (
    int((target_width - original_width * 0.5 ) * horizontal_position_percent),
    int((target_height - original_height * 0.5 ) * vertical_position_percent),
)

# Create an input image which contains the original image with an expanded
# canvas.
input_image = Image.new("RGB", (target_width, target_height), (235, 235, 235))
input_image.paste(original_image, position)
input_image.save(image_outpainting_target_image)

In [ ]:
%%time

outpainting_source_encoded_image = encode_image(image_outpainting_source_image)

    
# Generate image condition on reference image
body = json.dumps(
    {
        "taskType": "OUTPAINTING",
        "outPaintingParams": {
            "text": prompt,  # Required
            "image": outpainting_source_encoded_image,  # Required
            "maskPrompt": mask_prompt,  # One of "maskImage" or "maskPrompt" is required
            "outPaintingMode": "PRECISE",  # One of "PRECISE" or "DEFAULT"
        },
        "imageGenerationConfig": {
                "numberOfImages": 1,
                "seed": 42,
            }
        
    }
)


In [ ]:
%%time

generated_image_b64 = invoke_bedrock_model(boto3_bedrock_runtime_client, body, amazon_titan_image_model_id )
decode_image(generated_image_b64, image_outpainting_target_image)

print("\n\n")
images = [ image_outpainting_source_image , image_outpainting_target_image ]
headings =[ "Source Image" ,  "Generated Image" ]
plot_multiple_images(images, headings, cols=2)
print("\n\n")

# Section 8: Background removal

### You previously generated an image of an elephant in a beautifull landscape using a prompt. Now, you want to remove the background from the main object (elephant) and generate image of only the elephant.
### The original image is located at data/generated_image/generated_image_5.png.
### To achieve this, you will use the background removal technique.

In [ ]:
image_bckground_removal_source_image = "data/generated_image/generated_image_5.png"
image_bckground_removal_target_image = "data/generated_image/generated_image_background_removal.png"

In [ ]:
%%time

image_bckground_removal_source_encoded_image = encode_image(image_bckground_removal_source_image)

body = json.dumps({
    "taskType": "BACKGROUND_REMOVAL",
    "backgroundRemovalParams": {
        "image": image_bckground_removal_source_encoded_image,
    }
})

In [ ]:
%%time

generated_image_b64 = invoke_bedrock_model(boto3_bedrock_runtime_client, body, amazon_titan_image_model_id )
decode_image(generated_image_b64, image_bckground_removal_target_image)

print("\n\n")
images = [ image_bckground_removal_source_image , image_bckground_removal_target_image ]
headings =[ "Source Image" ,  "Generated Image" ]
plot_multiple_images(images, headings, cols=2)
print("\n\n")

# Section 9: Combination of Text and Image

### You want to interpret one image with text. For example, there is a image of a shape. You want to understand the type of shape with the text on the image.
### Even, you have an AWS architecture. You want to understand the architecture leveraging generative AI. 

In [ ]:
%%time 

filename = [ "rectangle_image_dimention.png" , "aws_architecture_glue_dq_pipeline.png" ]
images = [ filename[0] , filename[1] ]
headings =[ "1. Rectangle Image" ,  "2. AWS Architecture Image" ]
print("\n\n")
plot_multiple_images(images, headings, cols=2)
print("\n\n")


#### The code defines a function generate_message to interact with the AWS Bedrock runtime client for invoking an AI model using specified parameters such as model ID, messages, maximum tokens, temperature, and top-p sampling. It prepares the request in JSON format, invokes the model, and parses the response. In the main execution, it processes a list of image files by encoding each image as base64, passing it to the model as a message, and printing the generated response. Images are displayed using IPython's Image display functionality.

#### Source of aws_architecture_glue_dq_pipeline.png is https://aws.amazon.com/blogs/big-data/set-up-alerts-and-orchestrate-data-quality-rules-with-aws-glue-data-quality/

In [ ]:
%%time

def generate_message(boto3_bedrock_runtime_client, model_id, messages, max_tokens=512, top_p=1, temp=0.5, system=''):
    """
    Generate a message using the Bedrock runtime client.
    
    Args:
        boto3_bedrock_runtime_client: Boto3 Bedrock runtime client.
        model_id (str): Model ID to invoke.
        messages (list): List of messages to pass to the model.
        max_tokens (int): Maximum tokens for response.
        top_p (float): Top P sampling parameter.
        temp (float): Temperature for sampling.
        system (str): Optional system-level instructions.
    
    Returns:
        dict: Parsed response body from the model invocation.
    """
    try:
        # Prepare the request body
        body = json.dumps({
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": max_tokens,
            "messages": messages,
            "temperature": temp,
            "top_p": top_p,
            "system": system
        })

        # Invoke the model
        response = boto3_bedrock_runtime_client.invoke_model(body=body, modelId=model_id)
        response_body = json.loads(response.get('body').read())
        return response_body

    except botocore.exceptions.ClientError as e:
        print(f"Client error occurred: {e}")
        raise
    except json.JSONDecodeError as e:
        print(f"Failed to parse JSON response: {e}")
        raise
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        raise

# Main execution
try:
    
    for file in filename:
        # Define the message list
        message_list = [
            {
                "role": 'user',
                "content": [
                    {"type": "image", "source": {"type": "base64", "media_type": "image/png", "data": encode_image(file)}},
                ]
            }
        ]

        # Generate the response
        response = generate_message(
            boto3_bedrock_runtime_client=boto3_bedrock_runtime_client,
            model_id="anthropic.claude-3-sonnet-20240229-v1:0",
            messages=message_list,
            max_tokens=512,
            temp=0.5,
            top_p=0.9
        )
        
        from IPython.display import Image as IPImage

        print(f"Image of: {file}")
        print("\n\n")
        display(IPImage(file))
        print("\n\n")
        # Print the response text
        print(response['content'][0]['text'])
        print("\n\n")
        
except Exception as e:
    print(f"An error occurred during the main execution: {e}")

# End of NoteBook 

#### <ins>Step 1</ins> 

##### Please ensure that you close the kernel after using this notebook to avoid any potential charges to your account.

##### Process: Go to "Kernel" at top option. Choose "Shut Down Kernel". 
##### Refer https://docs.aws.amazon.com/sagemaker/latest/dg/studio-ui.html